#.**DATA UNDERSTANDING**


## **Context**

Sebuah perusahaan yang menyewakan DVD ingin mengetahui gambaran umum tentang bisnis yang sedang mereka jalankan. Terdapat sebuah database yang menunjukan aktivitas dari perusahaan seperti jumlah sewa, jumlah customer, total income per bulan pada setiap toko.

Dari database tersebut, fokus analisis kita ke tabel store & staff untuk mendapatkan insight yang dapat dijalankan.


## **Database Information**

Database yang dimiliki mempunyai 16 tabel, yaitu:

*   **Actor** : Informasi tentang data actor. 
*   **Address :** Informasi alamat untuk pelanggan, staf, dan toko.
*   **Category :** Kategori yang dapat ditetapkan ke sebuah film. 
*   **City :** Daftar kota. 
*   **Country** : Daftar negara.
*   **Customer :** Informasi tentang data pelanggan/customer. 
*   **Film** : Daftar semua film yang berpotensi tersedia di toko-toko. Salinan stok aktual dari setiap film disajikan dalam tabel inventory. 
*   **Film_actor** : Digunakan untuk mendukung many-to-many relationship antara film dan aktor. Untuk setiap aktor dalam film tertentu, akan ada satu baris di tabel film_actor yang mencantumkan aktor dan film. 
*   **Film_category :** Digunakan untuk mendukung many-to-many relationship antara film dan kategori. Untuk setiap kategori yang diterapkan pada sebuah film, akan ada satu baris dalam tabel kategori film yang mencantumkan kategori dan film. 
*   **Film_text :** Berisi kolom film_id, judul dan deskripsi dari tabel film, dengan isi tabel tetap sinkron dengan tabel film melalui triggers pada tabel film operasi INSERT, UPDATE dan DELETE. 
*   **Inventory :** Berisi satu baris untuk setiap salinan film tertentu di toko tertentu. 
*   **Language :** Tabel pencarian yang mencantumkan kemungkinan bahasa yang dimiliki film untuk bahasa dan nilai bahasa aslinya. a lookup table listing the possible languages that films can have for their language and original language values.
*   **Payment :** Mencatat setiap pembayaran yang dilakukan oleh pelanggan, dengan informasi seperti jumlah dan sewa yang dibayar (bila berlaku). 
*   **Rental :** Berisi satu baris untuk setiap penyewaan setiap item inventaris dengan informasi tentang siapa yang menyewa item apa, kapan disewa, dan kapan dikembalikan. 
*   **Staff :** Daftar semua anggota staf, termasuk informasi untuk alamat email, informasi login, dan gambar. 
*   **Store :** Daftar semua toko dalam sistem. Semua inventory ditugaskan ke toko tertentu, dan staf serta pelanggan diberi "toko rumah". 

Setiap tabel yang tertera pada database dapat terhubung, baik secara langsung maupun tidak langsung, sehingga setiap informasi dari database ini akan dapat saling berkaitan.


#**DATABASE**


## **Connecting To Database**

Bagian ini merupakan langkah awal untuk mulai melakukan proses analisis data. Pertama adalah membuat koneksi ke database di mana seperti yang sudah dijelaskan sebelumnya, database yang akan digunakan adalah database ``sakila``. Dengan melakukan koneksi ini, kita dapat mengakses seluruh tabel yang ada pada database.


In [ ]:
# Import Modules

import pandas as pd
import numpy as np
import mysql.connector as sqlcon

In [ ]:
# Connect To Database

mydb = sqlcon.connect(
    host = 'localhost',
    user = 'root',
    passwd = '262626',
    database = 'sakila'
)

Selain melakukan koneksi ke database, hal lain yang perlu dilakukan pada bagian awal ini adalah membuat sebuah fungsi untuk menuliskan ``query`` yang kemudian akan disimpan dalam bentuk DataFrame. Tujuannya adalah agar data hasil ``query`` yang dibuat dapat disimpan dan digunakan untuk keperluan analisis data. Nantinya, di sini terdapat 2 query untuk mengambil data-data yang ada di database, bukan hanya berasal dari 1 tabel, tapi juga akan melibatkan hubungan antar tabel.

In [ ]:
# Query Function

curs = mydb.cursor()

def sql_table (query) :
    curs.execute (query)
    result = curs.fetchall()
    df = pd.DataFrame(result, columns=curs.column_names)
    return df

## **Data Detail**
Data pertama ini merupakan data utama yang nantinya akan dianalisa lebih lanjut. Data ini merupakan gabungan dari 3 tabel, yaitu tabel ```Store```, ```Payment```, dan ```Rental```. Masing-masing dari setiap tabel tersebut diambil beberapa kolomnya dan tidak diambil secara keseluruhan. Informasi-informasi yang dianggap penting saja lah yang diambil. Informasi yang diambil antara lain adalah :
- store dari tabel store
- yearmonth dari tabel rental
- rental_date dari tabel rental
- return_date dari tabel rental
- total_rental dari tabel rental
- total_customer dari tabel customer
- total_income dari tabel payment 

In [ ]:
# Dataframe

df = sql_table (
'''
select s.store_id as store, 
DATE_FORMAT(r.rental_date, "%Y-%m") as yearmonth ,
rental_date, return_date,
count(*) as total_rental, 
count(distinct r.customer_id) as total_customer, 
round(sum(amount)) as total_income 
from rental r
join staff s
on s.staff_id = r.staff_id
join payment p
  on p.rental_id = r.rental_id
group by yearmonth , s.store_id
order by s.store_id, yearmonth;
'''
)
df.head(10)

### *Data jumlah sewa per bulan untuk setiap toko*


In [ ]:
# Total Rental per month (Aggregating)

tabel1 = sql_table (
    '''
select s.store_id as store, 
DATE_FORMAT(r.rental_date, "%Y-%m") as yearmonth ,
count(*) as total_rental
from rental r
join staff s
on s.staff_id = r.staff_id
group by yearmonth , s.store_id
order by s.store_id,yearmonth; 
    '''
)
tabel1.head(10)

### *Data jumlah customer per bulan untuk setiap toko*


In [ ]:
# Total Customer per month (Aggregating)

tabel2 = sql_table(
    '''
select c.store_id as store,
DATE_FORMAT(r.rental_date, "%Y-%m") as yearmonth ,
count(distinct r.customer_id) as total_customer 
from rental r
join customer c
on c.customer_id = r.customer_id
group by yearmonth , c.store_id
order by c.store_id, yearmonth; 
    '''
)

tabel2.head(10)


### *Data total pendapatan per bulan untuk setiap toko*

In [ ]:
# Total Income per month (Aggregating)

tabel3 = sql_table(
    '''
select s.store_id as store,
DATE_FORMAT(r.rental_date, "%Y-%m") as yearmonth ,
round(sum(amount)) as total_income 
from payment p
join rental r 
  on p.rental_id = r.rental_id 
join staff s
  on s.staff_id = r.staff_id
group by yearmonth , s.store_id
order by s.store_id, yearmonth;
'''
)
tabel3.head(10)

# **DATA MANIPULATION**

Seperti yang telah dijelaskan sebelumnya, data yang digunakan untuk dianalisis adalah data pada ```df```. Sebelum melakukan analisis lebih lanjut, hal yang harus dilakukan adalah mengecek informasi serta anomali pada data. Jika memang terdapat hal-hal yang dianggap 'kotor' pada data, maka yang perlu dilakukan adalah melakukan penanganan pada bagian tersebut. Pada bagian ini, data akan 'dibersihkan', sehingga output akhir yang diharapkan adalah terdapat sebuah dataset yang bersih yang dapat dianalisis lebih lanjut dengan menampilkan visualisasi, serta melihat statistics-nya.

## **Data Anomalies**

In [ ]:
# Check Info Tabel

tabel1.info()
tabel2.info()
tabel3.info()

## **Melihat Data Sekilas Dari General Info**

Jika melihat informasi tersebut, tidak semua kolom atau feature yang memiliki data lengkap, yaitu **return_date**. Selain dari itu, ada beberapa feature yang datanya juga missing, yang nantinya akan ditampilkan pada bagian berikutnya. **Kesimpulan pertama adalah bahwa terdapat *missing value* yang harus ditanggulangi.**

Fokus berikutnya adalah perhatikan pada features **total_income**

Feature ini merupakan feature yang seharusnya memiliki tipe data numerik (dibuktikan pada preview data di bagian sebelumnya), sedangkan yang terbaca tipe data dari feature ini adalah object. Artinya, feature ini tidak dianggap memiliki komponen data yang numerik. Tentu saja hal tersebut harus ditanggulangi, mengingat ke depannya data yang bersifat numerik ini akan digunakan.

Oleh karena itu, features ini juga harus ditanggulangi dengan cara mengubah tipe datanya. **Kesimpulan keduanya adalah terdapat features yang memiliki tipe data yang salah dan harus diubah sesuai dengan tipe data seharusnya.**

In [ ]:
# Check Missing Value Percentage

df.isnull().sum()

## **Missing Values**

Hanya feature return_date yang memiliki missing value sebanyak 2. Untuk mengatasinya cukup dengan menghapus 2 row tersebut, karna hanya sedikit maka tidak akan mengurangi informasi secara signifikan.

## **Handling Anomalies**

Setelah mengetahui hal yang harus 'dibereskan' terlebih dahulu, maka pada bagian ini, hal tersebut harus diatasi. Pertama, masalah missing value kita atasi dengan menghapus row yang berisi nilai null dengan dropna()

In [ ]:
# Remove Missing Value 
df.dropna(axis=0)

## **Mengubah Tipe Data Yang Salah**

Oke, telah disebutkan juga sebelumnya bahwa ada tipe data yang tidak sesuai. Features tersebut terlebih dahulu diubah agar fungsionalitasnya kembali ke hakekatnya. Numerik akan diperlakukan sebagai tipe data numerik. Tujuannya tentu saja agar features tersebut dapat dipergunakan sebagaimana mestinya.

In [ ]:
# Change Spesific Column To Numeric Format

df['total_income'] = pd.to_numeric(df['total_income'])
tabel3['total_income'] = pd.to_numeric(tabel3['total_income'])

In [ ]:
# Recheck Info

df.info()
tabel3.info()

## **Recheck Data Information**

Bagian sebelumnya, Feature **total_income** yang memiliki tipe data yang salah sudah diubah ke dalam tipe data yang seharusnya, yang awalnya bertipe data object sudah berubah menjadi float64. Untuk memastikannya, output di atas merupakan informasi umum yang kembali diperlihatkan untuk memastikan tipe data yang sudah diubah tersebut. 

Dengan begini, perubahan yang dilakukan sebelumnya sudah terimplementasi dan anomali yang kedua sudah teratasi. 

In [ ]:
# Check Duplicate

df[df.duplicated()]


## **Data Duplicate**

Anomali berikutnya yang bisa ditemui adalah data yang duplikat. Tentu saja data yang bersifat duplikat ini akan menjadi sesuatu hal yang akan mengganggu proses analisis data. Jika memang nantinya terdapat data yang duplikat, sebaiknya data duplikatnya dihapus dan disisakan data yang unique saja. Untuk data saat ini, melihat output di atas artinya tidak terdapat data yang duplikat. Dengan begitu tidak perlu ada action yang dilakukan.

## **Feature 'RentalDuration'**
Data awal menunjukan terdapat 2 features yang merupakan tipe data datetime. Artinya, kita dapat melakukan ekstraksi informasi tambahan dari kedua features tersebut. Sebelumnya, kita perlu tahu dulu definisi dari kedua tabel tersebut. **rental_date** adalah tanggal customer meminjam DVD, sedangkat **return_date** adalah tanggal customer mengembalikan DVD. 

Melihat kedua definisi tersebut, sebuah informasi dapat diambil, yakni seberapa lama waktu pinjam customer menyewa DVD. 

Oleh karena itu, untuk mendapatkan informasinya, maka perlu dilakukan pengurangan antara return_date dan juga rental_date. Mungkin akan timbul pertanyaan, apakah waktu dapat dikurangkan? Jawabannya, bisa. Output yang keluar nantinya akan berupa selisih atau lamanya waktu proses tersebut dalam satuan hari.

In [ ]:
# Add New Column (Lama Pinjam)

rental_duration = sql_table(
    '''
select rental_id, staff_id, customer_id , rental_date, 
return_date, DateDiff (return_date,rental_date) as lama_pinjam
from rental order by rental_id;
    '''
)
rental_duration.sample(5)

## **Unique Value 'RentalDuration'**
Berikut hasil yang memperlihatkan unique values beserta jumlah data di setiap unique values-nya.

In [ ]:
# Check Data Anomalies 

rental_duration['lama_pinjam'].value_counts() 

## **Anomali Pada RentalDuration**
Melihat output unique values beserta dengan banyaknya data di setiap unique values tersebut, terlihat tidak ada data yang salah, semua data waktu menunjukan tidak ada nilai minus


## **Preview Cleaned Data**

Di bawah ini adalah sample data yang dianggap sudah bersih setelah melewati proses-proses sebelumnya.

In [ ]:
# Clean Data

df.sample(5)

## **Total Pendapatan tiap Toko**


In [ ]:
# Groupping and Aggregating

df[['store','total_income']].groupby('store').describe()

## **Data Outlier**

In [ ]:
# Outlier Check With Function
Q1_amount = df['total_income'].describe()['25%']
Q3_amount = df['total_income'].describe()['75%']
iqr = Q3_amount - Q1_amount

outlier_index = df[(df['total_income'] < Q1_amount - (1.5 * iqr)) | (df['total_income']> Q3_amount + (1.5 * iqr)) ].index
not_outlier_index = df[(df['total_income'] > Q1_amount - (1.5 * iqr)) & (df['total_income']< Q3_amount + (1.5 * iqr)) ].index
df.loc[outlier_index]

Tidak terdapat outlier pada data ini.

# **DATA VISUALIZATION & STATISTICS**

Setelah mendapatkan data yang sudah 'bersih' dan siap untuk digunakan, masuklah kita ke bagian analisis data dengan menggunakan visual sebagai medianya. Di sini, kita akan melakukan visualisasi data untuk mendapatkan beberapa insight yang kemudian dapat menjadi landasan dalam pengambilan keputusan dan penyusunan strategi yang kuat untuk mendapatkan profit yang sebesar-besarnya dengan kerugian yang minim.

In [ ]:
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

Kita akan menampilkan grafik total customer dan total rental  tiap bulan untuk di analisis

### *Total Customer per month*

In [ ]:
#Total Customer per month each store

sns.barplot(data=df, x="yearmonth", y="total_customer", hue="store")
plt.title('Total Customer per month', size=15)
plt.legend(loc=0)
plt.show()

### *Total Rental per month*

In [ ]:
sns.barplot(data=df, x="yearmonth", y="total_rental", hue="store")
plt.title('Total Rental per month', size=15)
plt.legend(loc=0)
plt.show()

## **Profit Rental DVD**

In [ ]:
df['avg_payment'] = df['total_income'] / df['total_customer']
yearprof = df[['yearmonth', 'total_rental' ,'total_income','avg_payment']].groupby(pd.DatetimeIndex(df['yearmonth'])).sum()
yearprof

In [ ]:
# Line Plot Profit by month

plt.style.use('seaborn')
plt.figure(figsize=(20,10))
plt.plot(yearprof.index, yearprof['total_income'], 'ro-')
plt.title('Profit 2005 - 2006 Rental DVD', size = 20)
plt.xlabel('Bulan', size = 18)
plt.ylabel('Profit', size = 18)
plt.xticks(yearprof.index, rotation = 90, size = 12)
plt.yticks(rotation = 45, size = 12)

for x,y in zip(yearprof.index, round(yearprof['total_income'],1)) :
    plt.annotate(y,
    (x,y),
    textcoords = 'offset pixels',
    xytext = (1,15))

plt.show()

In [ ]:
# Line Plot Total Customer by month

plt.style.use('seaborn')
plt.figure(figsize=(20,10))
plt.plot(yearprof.index, yearprof['total_customer'], 'ro-')
plt.title('Total Customer 2005 - 2006 Rental DVD', size = 20)
plt.xlabel('Bulan', size = 18)
plt.ylabel('Total Customer', size = 18)
plt.xticks(yearprof.index, rotation = 90, size = 12)
plt.yticks(rotation = 45, size = 12)

for x,y in zip(yearprof.index, round(yearprof['total_customer'],1)) :
    plt.annotate(y,
    (x,y),
    textcoords = 'offset pixels',
    xytext = (1,15))

plt.show()

In [ ]:
# Line Plot Total Rental by month

plt.style.use('seaborn')
plt.figure(figsize=(20,10))
plt.plot(yearprof.index, yearprof['total_rental'], 'ro-')
plt.title('Total Rental 2005 - 2006 Rental DVD', size = 20)
plt.xlabel('Bulan', size = 18)
plt.ylabel('Total Rental', size = 18)
plt.xticks(yearprof.index, rotation = 90, size = 12)
plt.yticks(rotation = 45, size = 12)

for x,y in zip(yearprof.index, round(yearprof['total_rental'],1)) :
    plt.annotate(y,
    (x,y),
    textcoords = 'offset pixels',
    xytext = (1,15))

plt.show()

Database yang dimiliki saat ini adalah database Profit 2005 - 2006 Rental DVD. Mari coba kita lihat total profit yang didapatkan oleh perusahaan dari tahun ke tahun. 

Terlihat jelas pada grafik, penjualan terbesar secara total dalam periode ini terjadi pada bulan 7 tahun 2005

Secara grafik, dapat dilihat bahwa pada tahun 2006 nilai penjualan cukup menurun. Meski begitu, kita tidak bisa langsung menarik kesimpulan bahwa tahun 2006 ini tidak berjalan dengan baik masalah penjualannya. 

Hal tersebut dikarenakan dari database yang dimiliki, tahun 2006 ini belum sepenuhnya selesai. Grafik di bawah akan menjelaskan bahwa tahun 2005 berakhir sampai bulan ke 2 saja. Artinya, masih ada 10 bulan pada tahun tersebut, di mana datanya masih belum tersedia.

Pada bulan 6 tahun 2005, total customer bertambah tapi total rental dan profit berkurang.

Jika di lihat dari rata-rata biaya rental, pada bulan 6 tahun 2005, biaya rental nya tidak sebesar bulan 7 dan 8 tahun 2005, sehingga profit yang masuk tidak begitu besar.

## **Uji Perbandingan**

In [ ]:
# Uji Perbandingan Nilai Total Income

from scipy.stats import shapiro

norm, pval = shapiro(df['total_income'])

if pval < 0.05 :
    print (f'Tolak H0 Karena P-Value ({pval} < 5%)')
    print ('DATA TIDAK BERDISTRIBUS NORMAL')
else :
    print (f'Gagal Tolak H0 Karena P-Value ({pval} > 5%)')
    print ('DATA BERDISTRIBUS NORMAL')

## **Hubungan / Korelasi**

In [ ]:
# Korelasi

plt.figure(figsize=(10,10))
sns.heatmap(df[['store', 'total_income', 'total_rental', 'total_customer']].corr('spearman'), annot=True)

In [ ]:
# Korelasi Antara Profit dan Customer

plt.figure(figsize=(15,10))
sns.scatterplot(df['total_income'], df['total_rental'], df['total_customer'])